In [1]:
import numpy as np

# 02. 回归问题

## 2.3 线性模型实战
在介绍了用于优化𝑤和𝑏的梯度下降算法后，我们来实战训练单输入神经元线性模型。首先我们需要采样自真实模型的多组数据，对于已知真实模型的玩具样例(Toy Example)， 我们直接从指定的真实模型中直接采样：
+ $y = 1.477x + 0.089$

##### 1. 采样数据
为了能够很好地模拟真实样本的观测误差，我们给模型添加误差自变量$\epsilon$，它采样自均值为0，标准差为0.01的高斯分布：
+ $y = 1.477x + 0.089 + \epsilon, \epsilon \thicksim \mathcal{N}(0, 0.01^2)$

通过随机采样$n = 100$次，我们获得$n$个样本的训练数据集$\mathbb{D}^{\mathrm{train}}$，代码如下：

In [2]:
data = []# 保存样本集的列表

for i in range(100): # 循环采样100个点
    x = np.random.uniform(-10., 10.) # 随机采样输入x
    # 采样高斯噪声
    eps = np.random.normal(0., 0.01)
    # 得到模型的输出
    y = 1.477 * x + 0.089 + eps
    data.append([x, y]) # 保存样本点

data = np.array(data) # 转换为2D Numpy数组

##### 2. 计算误差
循环计算在每个点$(x^{(i)}, y^{(i)})$处的预测值与真实值之间差的平方并累加，从而获得训练集上的均方误差损失值。代码如下：

In [3]:
def mse(b, w, points):
    # 根据当前的 w,b 参数计算均方差损失
    totalError = 0
    # 循环迭代所有点
    for i in range(0, len(points)): 
        x = points[i, 0] # 获得i号点的输入x
        y = points[i, 1] # 获得i号点的输出y
        # 计算差的平方，并累加
        totalError += (y - (w * x + b)) ** 2
    # 将累加的误差求平均，得到均方差
    return totalError/float(len(points))

最后的误差和除以数据样本总数，从而得到每个样本上的平均误差。

##### 3. 计算梯度
根据之前介绍的梯度下降算法，我们需要计算出函数在每一个点上的梯度信息：$\displaystyle(\frac{\partial \mathcal{L}}{\partial w}, \frac{\partial \mathcal{L}}{\partial b})$。

我们来推导一下梯度的表达式，首先考虑$\frac{\partial \mathcal{L}}{\partial w}$，将均方差函数展开：

$\begin{align}\displaystyle \frac{\partial \mathcal{L}}{\partial w} &= \frac{\partial \frac{1}{n}\sum_{i=1}^{n}(wx^{(i)} + b - y^{(i)})^2}{\partial w} \\ &= \frac{1}{n}\sum_{i=1}^{n}\frac{\partial(wx^{(i)} + b - y^{(i)})^2}{\partial w}\\ &= \frac{1}{n}\sum_{i=1}^{n}2(wx^{(i)} + b - y^{(i)})\frac{\partial(wx^{(i)} + b - y^{(i)})}{\partial w}\\ &= \frac{2}{n}\sum_{i=1}^{n}2(wx^{(i)} + b - y^{(i)})x^{(i)}\end{align}$

用同样的方法，我们可以推导$\frac{\partial \mathcal{L}}{\partial b}$的偏导数：

$\begin{align}\displaystyle \frac{\partial \mathcal{L}}{\partial b} &= \frac{\partial \frac{1}{n}\sum_{i=1}^{n}(wx^{(i)} + b - y^{(i)})^2}{\partial b} \\ &= \frac{1}{n}\sum_{i=1}^{n}\frac{\partial(wx^{(i)} + b - y^{(i)})^2}{\partial b}\\ &= \frac{1}{n}\sum_{i=1}^{n}2(wx^{(i)} + b - y^{(i)})\frac{\partial(wx^{(i)} + b - y^{(i)})}{\partial b}\\ &= \frac{2}{n}\sum_{i=1}^{n}2(wx^{(i)} + b - y^{(i)})\end{align}$

根据偏导数的表达式，我们只需要计算在每一个点上面的$(wx^{(i)} + b − y^{(i)}) x^{(i)}$和$(wx^{(i)} + b − y^{(i)})$值，平均后即可得到偏导数$\displaystyle\frac{\partial \mathcal{L}}{\partial w}$和$\displaystyle\frac{\partial \mathcal{L}}{\partial b}$。实现如下：

In [4]:
def step_gradient(b_current, w_current, points, lr):
    # 计算误差函数在所有点上的导数，并更新 w,b
    b_gradient = 0
    w_gradient = 0
    # 总样本数
    M = float(len(points)) 
    for i in range(0, len(points)):
        x = points[i, 0]
        y = points[i, 1]
        b_gradient += (2/M) * ((w_current * x + b_current) - y)
        w_gradient += (2/M) * x * ((w_current * x + b_current) - y)
    # 根据梯度下降算法更新 w',b',其中 lr 为学习率
    new_b = b_current - (lr * b_gradient)
    new_w = w_current - (lr * w_gradient)
    return [new_b, new_w]

##### 4. 梯度更新
在计算出误差函数在$w$和$b$处的梯度后，我们可以更新$w$和$b$的值。我们把对数据集的所有样本训练一次称为一个`Epoch`，共循环迭代`num_iterations`个`Epoch`。实现如下：

In [5]:
def gradient_descent(points, starting_b, starting_w, lr, num_iterations):
    # 循环更新w,b多次
    b = starting_b # b 的初始值
    w = starting_w # w 的初始值
    # 根据梯度下降算法更新多次
    for step in range(num_iterations):
        # 计算梯度并更新一次
        b, w = step_gradient(b, w, np.array(points), lr)
        loss = mse(b, w, points) # 计算当前的均方差，用于监控训练进度
        if step%100 == 0: # 打印误差和实时的 w,b 值
            print(f"iteration:{step}, loss:{loss}, w:{w}, b:{b}")
    return [b, w] # 返回最后一次的w,b

主训练函数实现如下：

In [6]:
def main():
    lr = 0.01 # 学习率
    initial_b = 0 # 初始化b为0
    initial_w = 0 # 初始化w为0
    num_iterations = 1000
    # 训练优化 1000 次，返回最优 w*,b*和训练 Loss 的下降过程
    [b, w]= gradient_descent(data, initial_b, initial_w, lr, num_iterations)
    loss = mse(b, w, data) # 计算最优数值解 w,b 上的均方差
    print(f'Final loss:{loss}, w:{w}, b:{b}')

main()

iteration:0, loss:5.5559108101397365, w:1.0878737485740488, b:-0.01088794329648056
iteration:100, loss:0.00029992466589305237, w:1.4767873581796336, b:0.07466228872146469
iteration:200, loss:0.00010959378778883515, w:1.4769318134316611, b:0.08674836727243838
iteration:300, loss:0.00010617553247045912, w:1.4769511723442907, b:0.08836806150590652
iteration:400, loss:0.00010611414217195506, w:1.4769537666943953, b:0.08858512193612501
iteration:500, loss:0.00010611303963033697, w:1.4769541143715914, b:0.08861421090205028
iteration:600, loss:0.00010611301982919316, w:1.47695416096493, b:0.08861810920787117
iteration:700, loss:0.00010611301947357429, w:1.4769541672090531, b:0.08861863163236407
iteration:800, loss:0.00010611301946718586, w:1.476954168045848, b:0.08861870164414955
iteration:900, loss:0.00010611301946707576, w:1.4769541681579896, b:0.08861871102665395
Final loss:0.00010611301946706803, w:1.476954168172971, b:0.08861871228008317


上述例子比较好地展示了梯度下降算法在求解模型参数上的强大之处。

需要注意的是，对于复杂的非线性模型，通过梯度下降算法求解到的$w$和$b$可能是局部极小值而非全局最小值解，这是由模型函数的非凸性决定的。但是我们在实践中发现，通过梯度下降算法求得的数值解，它的性能往往都能优化得很好，可以直接使用求解到的数值解$w$和$b$来近似作为最优解。